In [ ]:
import time
import requests 
import hmac
import json
import hashlib
import numpy as np
from datetime import datetime
import pandas as pd
import gspread
import os
from datetime import datetime
#----------
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
#----------
from IPython.display import clear_output

# API info
API_HOST = 'https://api.bitkub.com'
API_KEY = config['API']['key']
API_SECRET = bytes(config['API']['secret'], 'utf-8') 

symbol = config['GRID_SYSTEM']['symbol']

In [ ]:
#-----------------------------------Bitkub---------------------------------------------#

In [ ]:
class bitkubAPI():
    def __init__(self,host,api_key,api_secret):
        self.API_HOST = host
        self.API_KEY = api_key
        self.API_SECRET = api_secret
        self.header = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'X-BTK-APIKEY': self.API_KEY,
            }
        
    def timestamp(self,tm):
        date = datetime.datetime.strptime(tm, "%Y-%m-%d")
        ts = time.mktime(date.timetuple())
        return ts
        
    def _json_encode(self,data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)

    def _sign(self,data):
        j = self._json_encode(data)
        h = hmac.new(API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
        return h.hexdigest()
    
    def _get(self,url,**kwargs): 
        try:
            res = requests.get(self.API_HOST+ url,**kwargs)
            res = json.loads(res.text)
            res = res
            return res
        except:
            return res
    
    def _post(self,url,data): 
        try:
            signature = self._sign(data)
            data['sig'] = signature
            res = requests.post(self.API_HOST + url, headers= self.header, data=self._json_encode(data))
            return res
        except:
            return res
        
    def api_status(self):  
        return self._get('/api/status')
        
    def serverTime(self):
        res = requests.get(self.API_HOST + '/api/servertime')
        return json.loads(res.text)
    
    def bids(self,sym):
        return self._get('/api/market/bids?sym='+sym+'&lmt=1')

    def asks(self,sym):
        return self._get('/api/market/asks?sym='+sym+'&lmt=1')
    
    def balance(self):
        data = {
            'ts': self.getServerTime(),
        }
        return self._post('/api/market/balances',data)
    
    def opened_orders(self,sym):
        data = {
            'sym': sym,
        }
        return self._post('/api/market/balances',data)
    
    def info_orders(self,sym,order_id,side,order_hash):
        data = {
            'sym': sym,
            'id':order_id,
            'sd':side,
            'hash':order_hash,
        }
        return self._post('/api/market/balances',data)
    
    def cancel_order(self,sym,order_id,side,order_hash):
        data = {
            'sym': sym,
            'id':order_id,
            'sd':side,
            'hash':order_hash,
        }
        return self._post('/api/market/cancel-order',data)
    
    def his_price(self,sym,frm):
        params = 'sym='+sym
        params = params + '&int=86400'
        params = params + f'&frm={int(self.timestamp(frm))}'
        params = '/api/market/tradingview?'+params
        return self._get(params)
    
    def place_order(self,sym,side,amt,rat,typ):
        data = {
            'sym': sym,
            'amt':amt,
            'rat':rat,
            'typ':typ,
        }
        if(side =='buy'):
            return self._post('/api/market/place-ask/test',data)
        elif(side =='sell'):
            return self._post('/api/market/place-bid/test',data)

In [ ]:
#-----------------------------------FTX---------------------------------------------#

In [ ]:
import time
import hmac
from typing import Optional, Dict, Any, List
from requests import Request, Session, Response

class ftxAPI():
    def __init__(self):
        self._api_host = 'https://ftx.com/api'
        self._api_key = b'1vyg6pC1WY-ODMhSyeHasYBV8ZVrgpAKYEu4uqWZ'
        self._api_secret  = 'qXuNtAcsQfZZEbzHlrm_rdBOtCrGgFHrYjdkshw8'
        self.ts = int(time.time() * 1000)
        self._subaccount_name = ""
        self._session = Session()
        
    def _get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('GET', path, params=params)
    
    def _post(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('POST', path, params=params)
    
    def _request(self, method: str, path: str, **kwargs) -> Any:
        request = Request(method, self._api_host + path, **kwargs)
        self._sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _sign_request(self, request: Request) -> None:
        self.ts = int(time.time() * 1000)
        prepared = request.prepare()
        signature_payload = f'{self.ts}{prepared.method}{prepared.path_url}'.encode()
        if prepared.body:
            signature_payload += prepared.body
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()
        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(self.ts)
        if self._subaccount_name:
            request.headers['FTX-SUBACCOUNT'] = urllib.parse.quote(self._subaccount_name)
    
    def _process_response(self, response: Response) -> Any:
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    
    
    def get_ticker(self,market_name):
        return self._get(f'/markets/{market_name}/orderbook?depth={1}')
    '''
        def place_orders(self,market,side,size,price,type_ord):
            return self._post(f'/orders',{   'market': market,
                                             'side': side,
                                             'price': price,
                                             'size': size,
                                             'type': type_ord,
                                         })
    '''

    
    def place_conditional_orders(self,market,side,size,type_ord,open_price,tp):
        return self._post(f'/conditional_orders',{    'market': market, 
                                                      'side': side,
                                                      'size': size,
                                                      'type': type_ord,
                                                      'triggerPrice':tp,
                                                      'orderPrice':open_price
                                                 })
    
    def historicalPrice(self,market_name,resolution,limit,start_time,end_time):
        return self._get(f'/markets/{market_name}/candles?resolution={resolution}&limit={limit}&start_time={start_time}&end_time={end_time}')
    
    def get_open_orders(self,market):
        return self._get(f'/orders?market={market}')
    
    def place_orders(self,market,side,size,price,type_ord):
        tm = time.localtime() # get struct_time
        time_string = time.strftime("%Y-%m-%d, %H:%M:%S", tm)
        if(side == 'buy'):
            price = self.get_ticker()['asks'][0][0]
        else:
            price = self.get_ticker()['bids'][0][0]
            
        return {"createdAt": time_string,
                "filledSize": 0,
                "future": market,
                "id": int(tm),
                "market": market,
                "price": 0.306525,
                "remainingSize": size,
                "side": side,
                "size": size,
                "status": "open",
                "type": type_ord,
              }
    
    def get_open_conditional_order(self,market):
        return self._get(f'/conditional_orders?market={market}')
    
    def get_conditional_order(self,conditional_order_id):
        return self._get(f'/conditional_orders/{conditional_order_id}/triggers')

In [ ]:
#--------------------------------------------------------------------------------#

In [ ]:
class symbol():
    def __init__(self,sym,API):
        self.symbol = sym
        self.ticker = {'ask':'','bid':'','askv':'','bidv':''}
        self.order = {}
        self.his_price={}
        self.API = API
        
        gc = gspread.service_account(os.getcwd()+'\\service_account.json')
        self.sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/16sEZIsXLYYOrjVKo8xLOpusOeeYGAmnw_Xnrm0y2qwg/edit?usp=drive_web&ouid=112544876186489015703')

        
    def get_ticker(self):
        try:
            ticker = self.API.get_ticker(self.symbol)
            self.ticker['bid'] = float(ticker['bids'][0][0])
            self.ticker['bidv'] = float(ticker['bids'][0][1])
            self.ticker['ask'] = float(ticker['asks'][0][0])
            self.ticker['askv'] = float(ticker['asks'][0][1])
            return True
        except:
            return False
            
    def load_order(self):
        try:
            worksheet = self.sheet.worksheets()
            #check sheet
            if (self.symbol not in str(worksheet)):
                #create sheert
                self.sheet.add_worksheet(title=f"{self.symbol}", rows="100", cols="13")
                self.sheet.worksheet(f"{self.symbol}").update('A1:O1', [['zone','status','open_id','open_date','open_price','side','size',
                                                                         'sl','tp','close_id','close_date','close_price','comment','profit']])
                print(f'----worksheet {self.symbol} create')

            if (f"{self.symbol}_history" not in str(worksheet)):
                self.sheet.add_worksheet(title=f"{self.symbol}_history", rows="100", cols="13")
                self.sheet.worksheet(f"{self.symbol}_history").update('A1:O1', [['zone','status','open_id','open_date','open_price','side','size',
                                                                         'sl','tp','close_id','close_date','close_price','comment','profit']])
                print(f'----worksheet {self.symbol}_history create')

            self.order = self.sheet.worksheet(f"{self.symbol}").get_all_records()
            self.summary = self.sheet.worksheet(f"summary").get_all_records()
            return True
        except:
            return False
        
    def create_log(self,res_order):
        #'symbol': 'THB_BTC', 'order_row': 2, 'history_row': 0
        self.summary = self.sheet.worksheet(f"summary").get_all_records()
        ord_row = self.summary['symbol']['order_row']+1
        his_row = self.summary['symbol']['history_row']+1

        if(status=='open'):
            self.sheet.worksheet(f"{self.summary['symbol']}").update(f'A{ord_row}:O{ord_row}',[res_order])
        
        elif(status=='close'):
            self.sheet.worksheet(f"{self.summary['symbol']}_history").update(f'A{his_row}:O{his_row}',[res_order])
            for i in range(len(self.order)):
                if(self.order[i]['zone'] == res_order['zone']
                  and self.order[i]['open_id']==res_order['open_id']):
                    self.sheet.worksheet.delete_row(i+1)
                    break
        
    def getHisPrice(self,tf,nbar):
        price = self.API.historicalPrice(self.symbol,tf,nbar,int(time.time())-(tf*nbar),int(time.time()))
        self.his_price = pd.DataFrame.from_dict(price)     
        
    def ma(self,type_bar):
        return self.his_price[type_bar].mean()

    def refBar(self,type_bar,bar):
        return self.his_price[type_bar][len(self.his_price)-bar]
    
    def atr(self):
        tr=[]
        for i in range(len(self.his_price)):
            tr.append(
                max(
                    (self.his_price['high'][i]-self.his_price['low'][i]),
                    abs(self.his_price['high'][i]-self.his_price['close'][i]),
                    (self.his_price['low'][i]-self.his_price['close'][i])
                )) 
        return np.mean(tr)
        

In [23]:
class main():
    def __init__(self,symlist,lot,margin):
        #self.API = bitkubAPI(API_HOST,API_KEY,API_SECRET)
        self.API = ftxAPI()
        self.summary=[]
        self.order = {}
        self.margin = margin
        self.lot = lot
        self.ma = 0
        self.atr = 1
        self.refBar = 0
        self.ticker = {}
        
        if len(symlist) == 1:
            self.symbol_0 = symbol(symlist[0],self.API)
            self.symbol_0.getHisPrice(300,100)
            self.ma = self.symbol_0.ma('close')
            self.refBar = self.symbol_0.refBar('high',1)
            self.zone = np.arange((self.ma//self.margin)*self.margin,((self.ma//self.margin)*self.margin)+(self.margin*self.atr*self.lot),self.margin) 
        if len(symlist) == 2:
            self.symbol_0 = symbol(symlist[0],self.API)
            self.symbol_0.getHisPrice(300,100)
    
            self.symbol_1 = symbol(symlist[1],self.API)
            self.symbol_1.getHisPrice(300,100)
            
            self.ma = self.symbol_0.ma('close') - self.symbol_1.ma('close') 
            self.refBar = self.symbol_0.refBar('high',1) - self.symbol_1.refBar('high',1)
            
        gc = gspread.service_account(os.getcwd()+'\\service_account.json')
        self.sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/16sEZIsXLYYOrjVKo8xLOpusOeeYGAmnw_Xnrm0y2qwg/edit?usp=drive_web&ouid=112544876186489015703')
        self.system = False
            
    def getOrder(self):
        if( self.symbol_0.load_order() and self.symbol_1.load_order()):
            for i in self.symbol_0.order:
                self.order[i['zone']] = {}
                self.order[i['zone']]['symbol_0'] = i
                for j in self.symbol_1.order():
                    if(j['zone']==i['zone']):
                        self.order[i['zone']]['symbol_1'] = j
                  
    def open_long(self):
        res1 = self.API.place_orders(self.symbol_0.symbol,'buy',self.lot,self.ticker['ask'] ,'market')
        res2 = self.API.place_orders(self.symbol_1.symbol,'sell',self.lot,self.ticker['bid'] ,'market')
        self.order[self.ticker['ask']]={
            'symbol_0':{
                'zone':self.ticker['ask'],
                'status':'open',
                'side':'buy',
                'open_id' : res1['result']['id'],
                'open_date': res1['result']['ts'],
                'size': res1['result']['amt'],
                'open_price': res1['result']['rat'],
                'sl': '',
                'tp': '',
                'fee':'',
                'close_id':'',
                'close_date':'',
                'close_price':'',
                'comment':'',
                'profit':''
                },
            'symbol_1':{
                'zone':self.ticker['ask'],
                'status':'open',
                'side':'sell',
                'open_id' : res2['result']['id'],
                'open_date': res2['result']['ts'],
                'size': res2['result']['amt'],
                'open_price': res2['result']['rat'],
                'sl': '',
                'tp': '',
                'fee':'',
                'close_id':'',
                'close_date':'',
                'close_price':'',
                'comment':'',
                'profit':''
            },
                                        }
        self.symbol_0.order.append( self.order[self.ticker['ask']]['symbol_0'])
        self.symbol_1.order.append( self.order[self.ticker['ask']]['symbol_1'])
        
        self.symbol_0.create_log(self.order[self.ticker['ask']]['symbol_0'])
        self.symbol_1.create_log(self.order[self.ticker['ask']]['symbol_1'])
                            
    def open_short(self):
        res1 = self.API.place_orders(self.symbol_0.symbol,'sell',self.lot,self.ticker['bid'] ,'market')
        res2 = self.API.place_orders(self.symbol_1.symbol,'buy',self.lot,self.ticker['ask'] ,'market')
        self.order[self.ticker['bid']]={
            'symbol_0':{
                'zone':self.ticker['bid'],
                'status':'open',
                'side':'sell',
                'open_id' : res1['result']['id'],
                'open_date': res1['result']['ts'],
                'size': res1['result']['amt'],
                'open_price': res1['result']['rat'],
                'sl': '',
                'tp': '',
                'fee':'',
                'close_id':'',
                'close_date':'',
                'close_price':'',
                'comment':'',
                'profit':''
                },
            'symbol_1':{
                'zone':self.ticker['bid'],
                'status':'open',
                'side':'buy',
                'open_id' : res2['result']['id'],
                'open_date': res2['result']['ts'],
                'size': res2['result']['amt'],
                'open_price': res2['result']['rat'],
                'sl': '',
                'tp': '',
                'fee':'',
                'close_id':'',
                'close_date':'',
                'close_price':'',
                'comment':'',
                'profit':''
            },
                                        }
        self.symbol_0.order.append( self.order[self.ticker['bid']]['symbol_0'])
        self.symbol_1.order.append( self.order[self.ticker['bid']]['symbol_1'])
        
        self.symbol_0.create_log(self.order[self.ticker['bid']]['symbol_0'])
        self.symbol_1.create_log(self.order[self.ticker['bid']]['symbol_1'])
    
    def close_order(self):
        for order in self.order:
            zone = order['zone']
            if(order['symbol_0']['side'] == 'buy'
              and self.ticker['bid'] >= order.key() + self.margin):
                res1 = self.API.place_orders(self.symbol_0.symbol,'sell',self.lot,self.ticker['bid'] ,'market')
                res2 = self.API.place_orders(self.symbol_1.symbol,'buy',self.lot,self.ticker['ask'] ,'market')
                
                self.order[zone]['symbol_0']['status'] = 'close'
                self.order[zone]['symbol_0']['close_id'] = res1["id"]
                self.order[zone]['symbol_0']['close_date'] = res1["createdAt"]
                self.order[zone]['symbol_0']['close_price'] = res1["price"]
                self.order[zone]['symbol_0']['profit'] = res1["price"] - self.order[zone]['symbol_0']['open_price']
       

                self.order[zone]['symbol_1']['status'] = 'close'
                self.order[zone]['symbol_1']['close_id'] = res2["id"]
                self.order[zone]['symbol_1']['close_date'] = res2["createdAt"]
                self.order[zone]['symbol_1']['close_price'] = res2["price"]
                self.order[zone]['symbol_1']['profit'] = self.order[zone]['symbol_0']['open_price'] - res1["price"]
                
                self.symbol_0.create_log(self.order[zone]['symbol_0'])
                self.symbol_1.create_log(self.order[zone]['symbol_1'])

                self.symbol_0.order.pop(self.order[zone]['symbol_0'])
                self.symbol_1.order.pop(self.order[zone]['symbol_1'])

                self.order.pop(zone)
                break
                    
            if(order['symbol_0']['side'] == 'sell' 
               and self.ticker['ask'] <= order.key() - self.margin):
                res1 = self.API.place_orders(self.symbol_0.symbol,'buy',self.lot,self.ticker['ask'] ,'market')
                res2 = self.API.place_orders(self.symbol_1.symbol,'sell',self.lot,self.ticker['bid'] ,'market')

                self.order[zone]['symbol_0']['status'] = 'close'
                self.order[zone]['symbol_0']['close_id'] = res1["id"]
                self.order[zone]['symbol_0']['close_date'] = res1["createdAt"]
                self.order[zone]['symbol_0']['close_price'] = res1["price"]
                self.order[zone]['symbol_0']['profit'] = self.order[zone]['symbol_0']['open_price'] - res1["price"]
       

                self.order[zone]['symbol_1']['status'] = 'close'
                self.order[zone]['symbol_1']['close_id'] = res2["id"]
                self.order[zone]['symbol_1']['close_date'] = res2["createdAt"]
                self.order[zone]['symbol_1']['close_price'] = res2["price"]
                self.order[zone]['symbol_1']['profit'] = res1["price"] - self.order[zone]['symbol_0']['open_price']
                
                self.symbol_0.create_log(self.order[zone]['symbol_0'])
                self.symbol_1.create_log(self.order[zone]['symbol_1'])

                self.symbol_0.order.pop(self.order[zone]['symbol_0'])
                self.symbol_1.order.pop(self.order[zone]['symbol_1'])

                self.order.pop(zone)
                break
                    
    def start(self):
        try:
            #get_ticker
            self.system = all([self.symbol_0.get_ticker(),self.symbol_1.get_ticker()])
            self.ticker['ask']  = round(self.symbol_0.ticker['ask'] - self.symbol_1.ticker['bid'], 5)
            self.ticker['bid']  = round(self.symbol_0.ticker['bid'] - self.symbol_1.ticker['ask'], 5)
            self.symbol_0.getHisPrice(300,100)
            self.symbol_1.getHisPrice(300,100)
            self.ma =  round(self.symbol_0.ma('close') - self.symbol_1.ma('close'),5)
            self.system = True
        except:
            self.system = False
        #get_time
        tm = time.localtime() # get struct_time
        time_string = time.strftime("%Y-%m-%d, %H:%M:%S", tm)

        if(self.system):
            #cal zone
            self.zone = round((self.ticker['ask'] // self.margin) * self.margin,5)
            
            #------open buy condition ------
            #------check time
            time_check = True #all([ tm.tm_min//5==0 , tm.tm_sec == 0])
            
            #------check long_conditon
            long_conditon = all([    self.ticker['ask'] <= self.ma,
                                   self.ticker['ask'] == self.zone,
                                   self.ticker['ask'] not in self.order.keys()
                              ])
            
            #------check short_conditon
            short_conditon = all([    self.ticker['bid'] >= self.ma,
                                   self.ticker['bid'] == self.zone,
                                   self.ticker['bid'] not in self.order.keys()
                              ])

            if(time_check and short_conditon):
                self.open_short()
            if(time_check and long_conditon):
                self.open_long()
            
            self.close_order()             
                        
            ask  = self.ticker['ask']
            
            print(f'{self.symbol_0.symbol}/{self.symbol_1.symbol} zone:{self.zone} ask:{ask} ma:{self.ma} {time_string}    ',end='\r')
        else:
            print(f'connection failed {time_string}                                                        ',end='\r')


In [26]:
program = main(['XRP-PERP','XLM-PERP'],1,0.0001)
program.getOrder()

In [30]:

while(True):
    clear_output(wait=True)
    program.start()


TypeError: get_ticker() missing 1 required positional argument: 'market_name'